In [1]:
def decompose(alpha,beta):
    return {(alpha,x) for x in set(beta)}

def isredundant(alpha,beta,F):
    return set(beta).issubset(set(attclosure(alpha,F-{(alpha,beta)})))

def removeredfd(F):
    F1 = F.copy()
    for alpha,beta in F:
        if isredundant(alpha,beta,F1):
            F1.discard((alpha,beta))
    return F1

def maxLHSextra(alpha,beta,F):
    X = powerset(set(alpha))
    X.remove(set())
    X.remove(set(alpha))
    ex = []
    for x in X:
        A = ''.join(x)
        if isLHSextra(alpha,beta,A,F):
            ex.append(x) 
    if len(ex)!= 0:
        ml = max(map(len,ex)) 
        ma = {''.join(x) for x in ex if len(x)==ml}
        return ma.pop()
    else:
        return ''

def ccover(F):
    Fc = set()
    for alpha,beta in F:
        Fc.update(decompose(alpha,beta))
    temp = Fc.copy()
    
    for alpha,beta in temp:
        A = set(maxLHSextra(alpha,beta,Fc))
        Fc.discard((alpha,beta))
        Fc.add((''.join(set(alpha) - A),beta))
    return removeredfd(Fc)

In [2]:
def powerset(s):
    if len(s)==1:
        return [set(),s]
    else:
        t = s.copy()
        a = t.pop()
        P = powerset(t)
        return P + [{a}|x for x in P]
    
def attclosure(alpha,F):
    result = alpha
    temp = ''
    while result!=temp:
        temp = result
        for x,y in F:
            if set(x).issubset(set(result)):
                result = ''.join(sorted(set(result) | set(y)))
    return result

def issuperkey(alpha,R,F):
    result = attclosure(alpha,F)
    return set(result).issuperset(set(R))


def findcandkey(R,F):
    P = powerset(set(R))
    P.remove(set())
    P.remove(set(R))
    ck = set()
    while len(P) > 0:
        alpha = ''.join(P.pop())
        if issuperkey(alpha, R, F):
            Q = powerset(set(alpha))
            Q.remove(set())
            Q.remove(set(alpha))
            tv = []
            while len(Q)>0:
                beta = ''.join(Q.pop())
                tv.append(issuperkey(beta, R, F))
            if not(any(tv)):
                ck.add(''.join(sorted(alpha)))
    return ck
            
    
def isvalidFD(alpha,beta,F):
    return set(beta).issubset(set(attclosure(alpha, F)))


def Fclosure(R,F):
    P = powerset(set(R))
    Fcl = F.copy()
    for x in P:
        if len(x) != 0:            
            alpha = ''.join(x)
            result = attclosure(alpha,F)
            Q = powerset(set(result))
            for y in Q:
                if len(y)!=0:
                    beta = ''.join(y)
                    Fcl.add((alpha,beta))
    return Fcl
            
def isLHSextra(alpha,beta,A,F):
    x = ''.join(set(alpha)-set(A))
    result = attclosure(x,F)
    return set(beta).issubset(set(result))

def isRHSextra(alpha,beta,A,F):
    x = ''.join(set(beta)-set(A))
    Fp = (F-{(alpha,beta)})|{(alpha,x)}
    result = attclosure(alpha,Fp)
    return set(A).issubset(set(result))

def isdeppres(alpha,beta,F,Decomp):
    result = alpha
    temp = ''
    while result!=temp:
        temp = result
        for R in Decomp:
            x =''.join(set(result) & set(R))
            xres = attclosure(x, F)
            t = set(xres) & set(R)
            result = ''.join(set(t)|set(result))
    return set(beta).issubset(set(result))
        
def isdeppresdecomp(Decomp,F):
    for alpha,beta in F:
        if not(isdeppres(alpha, beta, F, Decomp)):
            return False
    return True

def islossydecomp(R,F,Decomp):
    Ri = set()
    for r in Decomp:
        Ri.update(set(r))
    if Ri != set(R):
        print('Exit 1')
        return True 
    r1 = Decomp[0]
    for r2 in Decomp[1:]:
        alpha = ''.join(set(r1) & set(r2))
        if len(alpha) == 0:
            return True
        if not(issuperkey(alpha, r1, F) or issuperkey(alpha, r2, F)):
            return True
        r1 = ''.join(set(r1)|set(r2))
    return False

In [4]:
# 
R = 'ABCGHI'
FD = {('A', 'B'), ('G', 'CH'), ('BC', 'GI')}
findcandkey(R,FD)


{'AC', 'AG'}

In [ ]:
R = 'ABCDE'

FD = {('AB','C'),('DE','B'),('CD','E')}

findcandkey(R,FD)
{'ABD', 'ACD', 'ADE'}